Parte 1 – Filtrado y preprocesamiento

Cargue la información del archivo large_even.json en una lista, muestre la cantidad de
registros total (deben ser 746, 909). Este es nuestro tráfico inicial!

In [10]:
import json


total_records = 0
dns_events = []
all_events=[]


with open('large_eve.json', 'r') as file:
    for line in file:
        total_records += 1
        try:
            
            data = json.loads(line)
            all_events.append(data)
            if data.get('event_type') == 'dns':
                dns_events.append(data)
        
            # print(data)
        except json.JSONDecodeError as e:
            print(f"Error en la línea: {line}")
            print(f"Detalles del error: {e}")
print(f"Total de registros en el archivo: {total_records}")

Total de registros en el archivo: 746909


Debido a que estamos buscando dominios web, del total de registros, solamente estamos
interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS.

In [11]:

# Guardar los eventos DNS en un nuevo archivo JSON
with open('dns_events.json', 'w') as outfile:
    json.dump(dns_events, outfile, indent=4)

print(f"{len(dns_events)} eventos DNS guardados en 'dns_events.json'.")

15749 eventos DNS guardados en 'dns_events.json'.


Muestre la información de 2 registros cualesquiera

In [12]:
print(all_events[12])
print(all_events[20])

{'timestamp': '2017-07-22T17:33:17.786254-0500', 'flow_id': 1135816796360543, 'pcap_cnt': 30365, 'event_type': 'http', 'vlan': 140, 'src_ip': '192.168.204.59', 'src_port': 38012, 'dest_ip': '192.168.24.201', 'dest_port': 80, 'proto': 'TCP', 'tx_id': 1, 'http': {'hostname': '192.168.24.201', 'url': '/phppgadmin/browser.php', 'http_user_agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.15) Gecko/2009102814 Ubuntu/8.10 (intrepid) Firefox/3.0.15', 'http_content_type': 'text/html', 'http_refer': 'http://192.168.24.201/phppgadmin/', 'http_method': 'GET', 'protocol': 'HTTP/1.1', 'status': 200, 'length': 1125}}
{'timestamp': '2017-07-22T17:33:18.003156-0500', 'flow_id': 1135816796360543, 'pcap_cnt': 32000, 'event_type': 'http', 'vlan': 140, 'src_ip': '192.168.204.59', 'src_port': 38012, 'dest_ip': '192.168.24.201', 'dest_port': 80, 'proto': 'TCP', 'tx_id': 5, 'http': {'hostname': '192.168.24.201', 'url': '/phppgadmin/xloadtree/xloadtree2.js', 'http_user_agent': 'Mozilla/5.0 (X11; U; Li

Debido a que la data consiste en estructuras JSON anidadas, utilice la característica
json_normalize para normalizar la información y asignarla en un dataframe. Muestre el shape
del dataframe

In [15]:
from pandas import json_normalize

# Normalizar la data JSON anidada
df = json_normalize(all_events)

# Mostrar shape del DataFrame
print("Shape del DataFrame:", df.shape)

Shape del DataFrame: (746909, 199)


Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio)